# Fine-tuning Embedding Models

## Dependencies and Keys

### Nest Asyncio

In [1]:
import nest_asyncio

nest_asyncio.apply()

### Install Dependencies

In [2]:
!pip install -qU llama-index-llms-openai llama-index-embeddings-openai llama-index-finetuning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 68.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 41.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 51.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.5/375.5 kB 41.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 66.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 101.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [3]:
!pip install -qU llama-index-readers-file llama-index-embeddings-huggingface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 11.4 MB/s eta 0:00:00


In [4]:
!pip install -qU "sentence_transformers==2.7.0"

### Load Keys and Tokens

In [5]:
# load openai api key
import os
from getpass import getpass

os.environ['OPENAI_API_KEY'] = getpass('OpenAI API Key:')

OpenAI API Key:··········


In [6]:
# input hf token
from huggingface_hub import notebook_login

notebook_login()

## Loading Data

In [7]:
!pip install -qU gdown

Load training data

In [8]:
!gdown 1gaWqbHpKRM3M-h3MD_V5qjIaiR9T670J

Downloading...
From: https://drive.google.com/uc?id=1gaWqbHpKRM3M-h3MD_V5qjIaiR9T670J
To: /content/Insurance Policies Train.zip
100% 6.97M/6.97M [00:00<00:00, 17.0MB/s]


Load test data

In [9]:
!gdown 1ghcaOK33V9tWAeqkDCYMKB4cfvtRpckx

Downloading...
From: https://drive.google.com/uc?id=1ghcaOK33V9tWAeqkDCYMKB4cfvtRpckx
To: /content/Insurance Policies Test.zip
100% 1.51M/1.51M [00:00<00:00, 191MB/s]


Unzipping files...

In [10]:
!unzip "Insurance Policies Train.zip" -d ./train

Archive:  Insurance Policies Train.zip
  inflating: ./train/1 pds-pol-word-business-25-05-2012.pdf  
  inflating: ./train/__MACOSX/._1 pds-pol-word-business-25-05-2012.pdf  
  inflating: ./train/2 NMDMG10249.pdf  
  inflating: ./train/__MACOSX/._2 NMDMG10249.pdf  
  inflating: ./train/3 NHDHG6080.pdf  
  inflating: ./train/__MACOSX/._3 NHDHG6080.pdf  
  inflating: ./train/4 AU127-1.pdf   
  inflating: ./train/__MACOSX/._4 AU127-1.pdf  
  inflating: ./train/5 B2C_Comprehensive_Cert_EN_Nov_2014_Final.pdf  
  inflating: ./train/__MACOSX/._5 B2C_Comprehensive_Cert_EN_Nov_2014_Final.pdf  
  inflating: ./train/6 PARTICIPANT_GUIDE_INSURANCE.pdf  
  inflating: ./train/__MACOSX/._6 PARTICIPANT_GUIDE_INSURANCE.pdf  
  inflating: ./train/7 vero-enterprise-business-insurance-pds-policy-wording.pdf  
  inflating: ./train/__MACOSX/._7 vero-enterprise-business-insurance-pds-policy-wording.pdf  
  inflating: ./train/8 business-insurance-policy.pdf  
  inflating: ./train/__MACOSX/._8 business-insurance

In [11]:
!unzip "Insurance Policies Test.zip" -d ./test

Archive:  Insurance Policies Test.zip
  inflating: ./test/13 Home-insurance-policy-booklet-before-June-2012.PDF  
  inflating: ./test/__MACOSX/._13 Home-insurance-policy-booklet-before-June-2012.PDF  
  inflating: ./test/14 policy.pdf    
  inflating: ./test/__MACOSX/._14 policy.pdf  
  inflating: ./test/15 Barclays_BARTRVPD_07-16.pdf  
  inflating: ./test/__MACOSX/._15 Barclays_BARTRVPD_07-16.pdf  


In [12]:
from llama_index.core import SimpleDirectoryReader
from llama_index.core.node_parser import SimpleNodeParser

TRAIN_FILES = "train"
EVAL_FILES = "test"

def load_corpus(directory: str, verbose: bool = False) -> None:
    ### Verbose Mode
    if verbose: print(f"Loading files in {directory}")

    # load the data
    reader = SimpleDirectoryReader(directory)
    docs = reader.load_data()

    ### Verbose Mode
    if verbose: print(f"Loaded {len(docs)} docs")

    # parse the document into nodes
    parser = SimpleNodeParser.from_defaults()
    nodes = parser.get_nodes_from_documents(documents=docs, show_progress=verbose)

    ### Verbose Mode
    if verbose: print(f"Parsed {len(nodes)} nodes")

    return nodes

In [13]:
train_nodes = load_corpus(TRAIN_FILES, True)
eval_nodes = load_corpus(EVAL_FILES, True)

Loading files in train
Loaded 612 docs


Parsing nodes:   0%|          | 0/612 [00:00<?, ?it/s]

Parsed 641 nodes
Loading files in test
Loaded 104 docs


Parsing nodes:   0%|          | 0/104 [00:00<?, ?it/s]

Parsed 140 nodes


## Create a Fine-tuning Dataset

In [14]:
from llama_index.finetuning import generate_qa_embedding_pairs, EmbeddingQAFinetuneDataset
from llama_index.llms.openai import OpenAI

# instantiate llm
llm = OpenAI(model="gpt-3.5-turbo", temperature=0)

In [15]:
# create QA embedding pairs from train data
train_dataset = generate_qa_embedding_pairs(nodes=train_nodes[:100], llm=llm)
# save train dataset
train_dataset.save_json("train_dataset.json")

100%|██████████| 100/100 [02:45<00:00,  1.65s/it]


In [16]:
# create QA embedding pairs from eval data
eval_dataset = generate_qa_embedding_pairs(nodes=eval_nodes[:10], llm=llm)
# save eval dataset
eval_dataset.save_json("eval_dataset.json")

100%|██████████| 10/10 [00:14<00:00,  1.46s/it]


#### 🏁 Checkpoint - Load Datasets

In [17]:
from llama_index.finetuning import EmbeddingQAFinetuneDataset

# load datasets
train_dataset = EmbeddingQAFinetuneDataset.from_json("train_dataset.json")
eval_dataset = EmbeddingQAFinetuneDataset.from_json("eval_dataset.json")

## Fine-tuning `snowflake-arctic-embed-m`

In [24]:
from llama_index.finetuning import SentenceTransformersFinetuneEngine

# setup the fine-tuning engine
finetune_engine = SentenceTransformersFinetuneEngine(
    dataset=train_dataset,
    val_dataset=eval_dataset,
    model_id="Snowflake/snowflake-arctic-embed-m",
    model_output_path="snowflake_finetune_insurance",
    epochs=10
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/84.1k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/107 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Finally, let's fine tune the model!

In [25]:
# fine tune the embeddings
finetune_engine.finetune()

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Iteration:   0%|          | 0/20 [00:00<?, ?it/s]

Iteration:   0%|          | 0/20 [00:00<?, ?it/s]

Iteration:   0%|          | 0/20 [00:00<?, ?it/s]

Iteration:   0%|          | 0/20 [00:00<?, ?it/s]

...

Iteration:   0%|          | 0/20 [00:00<?, ?it/s]

Iteration:   0%|          | 0/20 [00:00<?, ?it/s]

Iteration:   0%|          | 0/20 [00:00<?, ?it/s]

Iteration:   0%|          | 0/20 [00:00<?, ?it/s]

Iteration:   0%|          | 0/20 [00:00<?, ?it/s]

In [26]:
# get fine-tuned model
finetuned_embedding = finetune_engine.get_finetuned_model()




Some weights of BertModel were not initialized from the model checkpoint at snowflake_finetune_insurance and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


#### 🏁 Checkpoint - Load Fine-tuned Embedding Model

In [27]:
from sentence_transformers import SentenceTransformer

fine_tuned_embedding = SentenceTransformer("snowflake_finetune_insurance")




Some weights of BertModel were not initialized from the model checkpoint at snowflake_finetune_insurance and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [29]:
# save model to hub
fine_tuned_embedding.save_to_hub("andre-fichel/snowflake-ft-insurance-m")

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

'https://huggingface.co/andre-fichel/snowflake-ft-insurance-m/commit/3326efd32cda0d158e701833a3e263277cad2ebd'

## Evaluating Retrieval with Embedding Model

### Benchmarking with OpenAI `text-embedding-small-3``

In [20]:
from tqdm.notebook import tqdm
from llama_index.core.schema import TextNode
from llama_index.core import Settings, VectorStoreIndex


def evaluate(dataset, embed_model, top_k=2, verbose=False):
    corpus = dataset["corpus"]
    queries = dataset["queries"]
    relevant_docs = dataset["relevant_docs"]

    nodes = [TextNode(id_=id_, text=text)
             for id_, text in corpus.items() if text != ""]
    index = VectorStoreIndex(nodes, show_progress=True,
                             embed_model=embed_model)
    retriever = index.as_retriever(similarity_top_k=top_k)

    eval_results = []
    for query_id, query in tqdm(queries.items()):
        retrieved_nodes = retriever.retrieve(query)
        retrieved_ids = [node.node.node_id for node in retrieved_nodes]
        expected_id = relevant_docs[query_id][0]
        is_hit = expected_id in retrieved_ids  # assume 1 relevant doc

        eval_result = {
            "is_hit": is_hit,
            "retrieved": retrieved_ids,
            "expected": expected_id,
            "query": query_id,
        }
        eval_results.append(eval_result)
    return eval_results

#### OpenAI `text-embedding-small-3`

In [18]:
# load eval dataset
import json

with open("eval_dataset.json", "r+") as f:
    eval_dataset_json = json.load(f)

In [21]:
# evaluate OpenAI embedding model
from llama_index.embeddings.openai import OpenAIEmbedding

text_embedding_3_small = OpenAIEmbedding(model="text-embedding-3-small")
te3_val_results = evaluate(eval_dataset_json, text_embedding_3_small)

Generating embeddings:   0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

In [38]:
# print the hit rate
from typing import List
import pandas as pd

def get_hit_rate(val_results: List) -> float:
    df = pd.DataFrame(val_results)
    return df["is_hit"].mean()

In [39]:
# te3 hit rate
hit_rate_te3 = get_hit_rate(te3_val_results); hit_rate_te3

1.0

#### Base `snowflake-arctic-embed-m`

In [31]:
# download base model
base_embed_model_id = "Snowflake/snowflake-arctic-embed-m"
base_embed_model = SentenceTransformer(base_embed_model_id)

In [32]:
# evaluate base arctic model
arctic_base = "local:Snowflake/snowflake-arctic-embed-m"
arctic_base_val_results = evaluate(eval_dataset_json, arctic_base)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/84.1k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/107 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/738 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Generating embeddings:   0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

In [40]:
# calculate hit rate
hit_rate_arctic_base = get_hit_rate(arctic_base_val_results); hit_rate_arctic_base

0.7

#### Fine-tuned Model

In [41]:
# evaluate base fine-tuned model
finetuned = "local:snowflake_finetune_insurance"
finetuned_val_results = evaluate(eval_dataset_json, finetuned)




Some weights of BertModel were not initialized from the model checkpoint at snowflake_finetune_insurance and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Generating embeddings:   0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

In [42]:
# calculate hit rate
hit_rate_finetuned = get_hit_rate(finetuned_val_results); hit_rate_finetuned

1.0

### Sentence Transformers Evaluation

In [53]:
from typing import Dict, Any
from sentence_transformers.evaluation import InformationRetrievalEvaluator
from sentence_transformers import SentenceTransformer


def evaluate_sentence_transformers(dataset: Dict[str, Any], model_id: str, name: str):
    corpus = dataset["corpus"]
    queries = dataset["queries"]
    relevant_docs = dataset["relevant_docs"]

    evaluator = InformationRetrievalEvaluator(queries, corpus, relevant_docs, name=name)
    model = SentenceTransformer(model_id)
    return evaluator(model, output_path="/content/")

#### Base Model

In [44]:
# evaluate base model
evaluate_sentence_transformers(
    eval_dataset_json, "Snowflake/snowflake-arctic-embed-m", name="arctic-m"
)

0.5304761904761903

#### Fine-tuned Model

In [45]:
evaluate_sentence_transformers(
    eval_dataset_json, "snowflake_finetune_insurance", name="finetuned"
)




Some weights of BertModel were not initialized from the model checkpoint at snowflake_finetune_insurance and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


0.95